# Point Clouds Processing

## Import libraries

In [1]:
import pyvista as pv
import glob
import laspy as lp
import numpy as np
from sklearn.neighbors import KDTree
import matplotlib.pyplot as plt

## Search for path of interest

In [ ]:
main_path = '/home/diego/workspace/SQM/SQM_project/Day 1 Pools 120m 50&100kHz/09_EXPORT/*.las'
#main_path = '/home/diego/workspace/SQM/SQM_project/Day 5 Big Pools 100kHz 220m/*'
paths = sorted(glob.glob(main_path))
paths

## Load and subsampling (if necessary) point cloud

In [3]:
def get_points_and_colors(las_paths: list, limit: int = -1):
    '''
    Get the points and colors from a list of las files. 
    If limit is set to -1, all points are returned.
    Otherwise, limit points are randomly selected.

    Parameters:
    las_paths: list
        List of paths to the las files.
    limit: int
        Number of points to be selected. Default is -1.
    '''
    points = []
    colors = []
    classes = []
    for las_path in las_paths:
        las = lp.read(las_path)
        print(f"Number of points for {las_path}: ", len(las))
        print(f"Classifications: {np.unique(las.classification)}")
        
        points.append(np.vstack((las.x, las.y, las.z)).transpose())
        colors.append(np.vstack((las.red, las.green, las.blue)).transpose())
        classes.append(las.classification)
    
    points = np.vstack(points)
    colors = np.vstack(colors)
    classes = np.vstack(classes)

    if limit > 0:
        idxs = np.random.choice(len(points), limit, replace=False)
        points = points[idxs]
        colors = colors[idxs]
        classes = classes[idxs]
        
    print("Number of points after selection: ", len(points))
    # normalize colors
    rgba = colors - colors.min(axis=0)
    # set between 0 and 1
    rgba = rgba / rgba.max(axis=0)
    # set between 0 and 255
    rgba_colors = (rgba * 255).astype(np.uint8)
    return points, colors, rgba_colors, classes

In [ ]:
points, colors, rgba_colors, classes = get_points_and_colors([paths[0]], limit=-1)

In [5]:
point_cloud = pv.PolyData(points)

In [ ]:
# Add the RGBA colors to the point cloud as a scalar array
point_cloud["colors"] = rgba_colors

# Create the PyVista plotter
plotter = pv.Plotter()

# Add aligned coordinate axes
plotter.add_axes()

# Add the point cloud with colors
plotter.add_mesh(point_cloud, scalars="colors", rgb=True, pickable=True)

# List to store picked points
picked_points = []

# Callback function to handle point picking
def callback(point):
    # Add the picked point to the list
    picked_points.append(point)

    # If two points are picked, create a perpendicular plane
    if len(picked_points) == 2:
        # Extract the two points
        p1, p2 = picked_points

        # Compute the normal vector of the plane (vector between the points)
        normal = np.array(p2) - np.array(p1)
        # Rotate the normal vector by 90 degrees to get a perpendicular vector
        normal = np.array([normal[1], -normal[0], normal[2]])
        normal = normal / np.linalg.norm(normal)  # Normalize the vector

        # Compute the midpoint between the two points (center of the plane)
        midpoint = (np.array(p1) + np.array(p2)) / 2

        # Define the plane using PyVista's pv.Plane
        plane = pv.Plane(center=midpoint, direction=normal, i_size=100, j_size=1000)

        # Add the plane to the plotter
        plotter.add_mesh(plane, color="red", opacity=0.5, name="picked_plane")

        # Update the render window to display the new plane
        plotter.render()

        # Reset picked points for the next selection
        #picked_points.clear()

# Enable point picking and assign the callback
plotter.enable_point_picking(callback=callback, show_message=True)

# Show the plotter
plotter.show()

In [ ]:
# List to store picked points
picked_points

In [ ]:
# Tolerance
tolerance = 2

# Project only in X and Y dimensions
point1_xy = picked_points[1][:2]
point2_xy = picked_points[0][:2]
point_cloud_xy = point_cloud.points[:, :2]

# Vector of the line segment in the XY-plane
line_vector_xy = point2_xy - point1_xy
line_length_xy = np.linalg.norm(line_vector_xy)
line_unit_vector_xy = line_vector_xy / line_length_xy

# Calculate projection of each point onto the line (XY-plane)
point_to_p1_vectors_xy = point_cloud_xy - point1_xy
projections_xy = np.dot(point_to_p1_vectors_xy, line_unit_vector_xy)

# Clamp projections to the segment range [0, line_length_xy]
clamped_projections_xy = np.clip(projections_xy, 0, line_length_xy)

# Find the closest point on the line for each point (XY-plane)
closest_points_on_line_xy = point1_xy + clamped_projections_xy[:, np.newaxis] * line_unit_vector_xy

# Calculate perpendicular distances in the XY-plane
distances_xy = np.linalg.norm(point_cloud_xy - closest_points_on_line_xy, axis=1)

# Mask: points within the tolerance in XY-plane
mask = distances_xy <= tolerance

# Extract points (including all Z-values for selected points)
diagonal_points = point_cloud.points[mask]
print(f"Number of points in the box: {len(diagonal_points)}")
new_point_cloud = pv.PolyData(diagonal_points)
new_point_cloud["colors"] = rgba_colors[mask]

# Create the PyVista plotter
plotter = pv.Plotter()

# Add the point cloud with a clip box, using the colors
plotter.add_mesh(new_point_cloud, scalars="colors", rgb=True, pickable=True)

# Show the plot
plotter.show()

In [ ]:
plt.figure(figsize=(20, 10))
selected_points = diagonal_points
selected_colors = rgba_colors[mask]
plt.scatter(selected_points[:, 1]*-1, selected_points[:, 2], c=selected_colors/255, s=25)
plt.xlabel('Valores eje Y')
plt.ylabel('Valores eje Z')
plt.title('Corte transversal de la poza')
# Adjust the aspect ratio of the plot to make sure both axes are proportional
plt.gca().set_aspect(25)
plt.show()

In [ ]:
new_points = point_cloud.points
# extract all points with np.mean(x)
median_x = np.median(new_points[:, 0])
# select indexes by two conditions
selected_indexes = np.where((new_points[:, 0] > median_x - 0.5) & (new_points[:, 0] < median_x + 0.5))[0]
selected_points = new_points[selected_indexes]
selected_colors = rgba_colors[selected_indexes]
# plot selected points in scatter 2d plot in real axis values
plt.scatter(selected_points[:, 1], selected_points[:, 2], c=selected_colors/255)
plt.xlabel('Y')
plt.ylabel('Z')
plt.show()
#picked_points = []

In [ ]:
import numpy as np
import pyvista as pv
from sklearn.decomposition import PCA


class PCARoutine:
    def __init__(self, points):
        self.original_points = points
        self.points = points  # Points to visualize
        self.output = pv.PolyData(points)
        self.kwargs = {
            'scale': 1.0,
            'rotate_x': 0.0,
            'rotate_y': 0.0,
            'rotate_z': 0.0,
        }
        self.update()

    def __call__(self, param, value):
        self.kwargs[param] = value
        self.update()

    def update(self):
        # Step 1: Translate the point cloud to align its centroid with the origin
        centroid = np.mean(self.original_points, axis=0)
        points_centered = self.original_points - centroid

        # Step 2: Apply scaling
        points_scaled = points_centered * self.kwargs['scale']

        # Step 3: Use PCA for alignment
        pca = PCA(n_components=3)
        pca.fit(points_scaled)
        rotation_matrix = pca.components_.T

        # Rotate points using updated angles
        angles = np.radians([
            self.kwargs['rotate_x'],
            self.kwargs['rotate_y'],
            self.kwargs['rotate_z'],
        ])
        rx = np.array([[1, 0, 0],
                       [0, np.cos(angles[0]), -np.sin(angles[0])],
                       [0, np.sin(angles[0]), np.cos(angles[0])]])
        ry = np.array([[np.cos(angles[1]), 0, np.sin(angles[1])],
                       [0, 1, 0],
                       [-np.sin(angles[1]), 0, np.cos(angles[1])]])
        rz = np.array([[np.cos(angles[2]), -np.sin(angles[2]), 0],
                       [np.sin(angles[2]), np.cos(angles[2]), 0],
                       [0, 0, 1]])

        rotation_matrix_combined = rz @ ry @ rx
        points_rotated = points_scaled @ rotation_matrix_combined

        # Update the PyVista mesh
        self.points = points_rotated
        self.output.points = points_rotated

# Create the PCA routine
engine = PCARoutine(local_maxima)

# Visualization
p = pv.Plotter()
engine.output["colors"]= rgba_colors
 # Add the point cloud with a clip box, using the colors
p.add_mesh_clip_box(engine.output, scalars="colors", rgb=True)
#p.add_mesh(engine.output, render_points_as_spheres=True, point_size=8, color='blue')

# Add sliders
# p.add_slider_widget(
#     callback=lambda value: engine('scale', value),
#     rng=[0.5, 2.0],
#     value=1.0,
#     title="Scale",
#     pointa=(0.025, 0.1),
#     pointb=(0.31, 0.1),
#     style='modern',
# )
p.add_slider_widget(
    callback=lambda value: engine('rotate_x', value),
    rng=[-180, 180],
    value=0,
    title="Rotate X",
    pointa=(0.025, 0.1),
     pointb=(0.31, 0.1),
    style='modern',
)
p.add_slider_widget(
    callback=lambda value: engine('rotate_y', value),
    rng=[-180, 180],
    value=0,
    title="Rotate Y",
    pointa=(0.35, 0.1),
    pointb=(0.64, 0.1),
    style='modern',
)
p.add_slider_widget(
    callback=lambda value: engine('rotate_z', value),
    rng=[-180, 180],
    value=0,
    title="Rotate Z",
    pointa=(0.67, 0.1),
    pointb=(0.96, 0.1),
    style='modern',
)

p.show()

In [112]:
# Create a new point cloud with the selected points
point_cloud["colors"] = rgba_colors
filtered_point_cloud = point_cloud.extract_points(p.box_clipped_meshes[0].cells)

In [ ]:
np.where(classes==65)

In [ ]:
point_cloud.points.shape

In [ ]:
# water points are classified as 65
water_points = point_cloud.points[np.where(classes == 65)]
#water_colors = rgba_colors[np.where(classes == 65)]
water_cloud = pv.PolyData(water_points)
#water_cloud['colors'] = water_colors
# underwater points are classified as 66
underwater_points = point_cloud.points[np.where(classes == 66)]
#underwater_colors = rgba_colors[np.where(classes == 66)]
underwater_cloud = pv.PolyData(underwater_points)
#underwater_cloud['colors'] = underwater_colors

In [ ]:
new_plotter = pv.Plotter()
new_plotter.add_mesh(filtered_point_cloud, show_edges=False, scalars="colors", rgb=True)
new_plotter.show()

In [ ]:
filtered_point_cloud

In [118]:
local_z_points = filtered_point_cloud.points[:,2]

In [119]:
import numpy as np
from sklearn.mixture import GaussianMixture

def find_gaussian_components(data, max_components=10, threshold=1e-3):
    """
    Quickly find the Gaussian distributions that compose a given distribution.
    
    Args:
        data (np.ndarray): Input data (1D or 2D array).
        max_components (int): Maximum number of components to test.
        threshold (float): Minimum weight threshold to filter out negligible components.
    
    Returns:
        dict: A dictionary containing the means, covariances, and weights of the Gaussian components.
    """
    # Fit Gaussian Mixture Model (GMM)
    gmm = GaussianMixture(n_components=max_components, random_state=42).fit(data.reshape(-1, 1))
    
    # Filter components with significant weights
    significant_indices = gmm.weights_ > threshold
    means = gmm.means_[significant_indices].flatten()
    covariances = gmm.covariances_[significant_indices].flatten()
    weights = gmm.weights_[significant_indices]

    # sort by means in ascending order
    sorted_indices = np.argsort(means)
    means = means[sorted_indices]
    covariances = covariances[sorted_indices]
    weights = weights[sorted_indices]
    
    # Return significant components
    return {
        "means": means,
        "covariances": covariances,
        "weights": weights
    }


In [120]:
data = local_z_points

In [121]:
components = find_gaussian_components(data, max_components=2, threshold=0.1)

In [ ]:
components

In [123]:
def find_percentile_1_50_99(mean, covariance):
    """
    Find the 1st, 50th, and 99th percentiles of a Gaussian distribution.
    
    Args:
        mean (float): Mean of the Gaussian distribution.
        covariance (float): Covariance of the Gaussian distribution.
        weight (float): Weight of the Gaussian distribution.
    
    Returns:
        tuple: The 1st, 50th, and 99th percentiles.
    """
    # Calculate the standard deviation
    std = np.sqrt(covariance)
    
    # Calculate the percentiles
    p1 = mean - 2.33 * std
    p2 = mean
    p3 = mean + 2.33 * std
    
    return p1, p2, p3

In [124]:
upper_index = np.argmax(components['means'])
upper_point = components['means'][upper_index]
upper_covariance = components['covariances'][upper_index]
upper_percentiles = find_percentile_1_50_99(upper_point, upper_covariance)
lower_index = np.argmin(components['means'])
lower_point = components['means'][lower_index]
lower_covariance = components['covariances'][lower_index]
lower_percentiles = find_percentile_1_50_99(lower_point, lower_covariance)

In [ ]:
upper_percentiles, lower_percentiles

In [ ]:
plt.hist(data)

In [ ]:
x = np.linspace(data.min(), data.max(), 1000).reshape(-1, 1)
plt.figure(figsize=(8, 5))
plt.hist(data, bins=100, density=True, alpha=0.5, label='Data')
for i, (mean, covariance, weight) in enumerate(zip(components['means'], components['covariances'], components['weights'])):
    gaussian = weight * (1 / np.sqrt(2 * np.pi * covariance)) * np.exp(-0.5 * ((x - mean) ** 2) / covariance)
    plt.plot(x, gaussian, label=f'Mean: {mean:.2f}')
# plot vlines in the 1st, 50th, and 99th percentiles of the upper and lower components
#plt.axvline(upper_percentiles[0], color='orange', linestyle='dashed', linewidth=1)
#plt.axvline(upper_percentiles[1], color='orange', linestyle='dashed', linewidth=1)
plt.axvline(upper_percentiles[2], color='g', linestyle='dashed', linewidth=1)
plt.axvline(lower_percentiles[0], color='orange', linestyle='dashed', linewidth=1)
plt.axvline(lower_percentiles[1], color='orange', linestyle='dashed', linewidth=1)
plt.axvline(lower_percentiles[2], color='orange', linestyle='dashed', linewidth=1)
print(upper_percentiles[0], upper_percentiles[1], upper_percentiles[2])
print(lower_percentiles[0], lower_percentiles[1], lower_percentiles[2])
plt.title("Gaussian Components")
plt.xlabel("Z values")
plt.ylabel("Density")
plt.legend()
plt.show()

In [ ]:
# get the diff between the 99th percentile of the upper and 1st percentile of the lower
diff_1 = upper_percentiles[2] - lower_percentiles[0]
print("Difference between the 99th percentile of the upper and 1st percentile of the lower: ", np.round(diff_1*100, 2), "cm")
# get the diff between the 99th percentile of the upper and 50th percentile of the lower
diff_2 = upper_percentiles[2] - lower_percentiles[1]
print("Difference between the 99th percentile of the upper and 50th percentile of the lower: ", np.round(diff_2*100, 2), "cm")
# get the diff between the 99th percentile of the upper and 99th percentile of the lower
diff_3 = upper_percentiles[2] - lower_percentiles[2]
print("Difference between the 99th percentile of the upper and 99th percentile of the lower: ", np.round(diff_3*100, 2), "cm")
mean = (diff_2 + diff_3) / 2
print("Mean: ", np.round(mean*100, 2), "cm")

In [ ]:
(upper_percentiles[2]-lower_percentiles[2])*100